In [1]:
from ortools.linear_solver import pywraplp as glp

In [2]:
# input parameters
auction = ['A1','A2', 'A3']
shops = ['S1', 'S2']      # list of suppliers (sources)
lots = ['L1', 'L2', 'L3', 'L4']  
buys = 15
sells = 10
cost_in = [[550, 500],     
           [600, 450],                    
           [525, 525]]   
cost_out = [[250, 300, 500, 400],  
            [350, 650, 450, 450]]  

In [3]:
# initialize LP model object
mymodel = glp.Solver('RVW', glp.Solver.GLOP_LINEAR_PROGRAMMING)


In [4]:
# create two dimensional lists of shipment variables
ship_in = list(range(len(auction)))
for i in ship_in:
    ship_in[i] = list(range(len(shops)))
    for j in ship_in[i]:
        ship_in[i][j] = mymodel.NumVar(0, buys, auction[i] +'-'+ shops[j])

print (ship_in)

ship_out = list(range(len(shops)))
for j in ship_out:
    ship_out[j] = list(range(len(lots)))
    for k in ship_out[j]:
            ship_out[j][k] = mymodel.NumVar(0, sells, shops[j] +'-'+ lots[k])
            
print(ship_out)

[[A1-S1, A1-S2], [A2-S1, A2-S2], [A3-S1, A3-S2]]
[[S1-L1, S1-L2, S1-L3, S1-L4], [S2-L1, S2-L2, S2-L3, S2-L4]]


In [5]:
# define objective function
TotCost = mymodel.Objective()
TotCost.SetMinimization()

# set ship_in costs
for i in range(len(ship_in)):
    for j in range(len(ship_in[i])):
        TotCost.SetCoefficient(ship_in[i][j], cost_in[i][j])

# set ship_out costs
for j in range(len(ship_out)):
    for k in range(len(ship_out[j])):
        TotCost.SetCoefficient(ship_out[j][k], cost_out[j][k])

In [6]:
# define supplier supply constraints
supply_constr = mymodel.Constraint(buys, buys)
for i in range(len(auction)):
    for j in range(len(shops)):
        supply_constr.SetCoefficient(ship_in[i][j], 1)

In [7]:
# define customer demand constraints
demand_constr = mymodel.Constraint(sells, sells)
for j in range(len(shops)):
    for k in range(len(lots)):
        demand_constr.SetCoefficient(ship_out[j][k], 1)


In [8]:
# define flow constraints
flow_constr = list(range(len(shops)))
for j in range(len(flow_constr)):
    flow_constr[j] = mymodel.Constraint(0, mymodel.infinity())
    for i in range(len(auction)):
        flow_constr[j].SetCoefficient(ship_in[i][j], 1)

    for k in range(len(lots)):
        flow_constr[j].SetCoefficient(ship_out[j][k], -1)

In [9]:
# Solve the model and print optimal solution
status = mymodel.Solve()                 # solve mymodel and display the solution

print('Solution Status =', status)
print('Number of variables =', mymodel.NumVariables())
print('Number of constraints =', mymodel.NumConstraints())

print('Optimal Solution:')

# The objective value of the solution.
print('Total Cost = %.2f' % TotCost.Value())

Solution Status = 0
Number of variables = 14
Number of constraints = 4
Optimal Solution:
Total Cost = 10000.00


In [10]:
# Display the shipments
print ('Shipments from auctions to shops')
for i in range(len(auction)):
    for j in range(len(shops)):
        if ship_in[i][j].solution_value():
            print('%10s to %10s \t %5.1f' % (auction[i], shops[j], ship_in[i][j].solution_value()))
        
print ('\nShipments from shops to lots')       
for j in range(len(shops)):
    for k in range(len(lots)):
        if ship_out[j][k].solution_value():
            print('%10s to %10s \t %5.1f' % (shops[j],lots[k], ship_out[j][k].solution_value()))
            

Shipments from auctions to shops
        A2 to         S2 	   5.0
        A3 to         S1 	  10.0

Shipments from shops to lots
        S1 to         L1 	  10.0


In [11]:
#b. Since the cars shipped from A2 only goes to S2 (which is also the cheapest)
# S1-L1 is costing 250 per car
# There is no lots where S2 could sell to that cost less than S1-L1
# Therefore, there's no value to an additional purchase at A2